In [1]:
import stanza
import xml.etree.ElementTree as ET
from nltk.tree import Tree
from dep_func import depTripleFuncLex
import json
import tqdm
# nlp_constituency = stanza.Pipeline('zh', processors='tokenize,pos,constituency')
nlp_dependency = stanza.Pipeline('zh')

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-09-17 11:07:20 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-09-17 11:07:21 INFO: Downloaded file to C:\Users\Administrator\stanza_resources\resources.json
2024-09-17 11:07:21 INFO: "zh" is an alias for "zh-hans"
2024-09-17 11:07:22 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package          |
-----------------------------------
| tokenize     | gsdsimp          |
| pos          | gsdsimp_charlm   |
| lemma        | gsdsimp_nocharlm |
| constituency | ctb-51_charlm    |
| depparse     | gsdsimp_charlm   |
| sentiment    | ren_charlm       |
| ner          | ontonotes        |

2024-09-17 11:07:22 INFO: Using device: cuda
2024-09-17 11:07:22 INFO: Loading: tokenize
2024-09-17 11:07:23 INFO: Loading: pos
2024-09-17 11:07:23 INFO: Loading: lemma
2024-09-17 11:07:23 INFO: Loading: constituency
2024-09-17 11:07:24 INFO: Loading: depparse
2024-09-17 11:07:24 INFO: Loading: sentiment
2024-09-17 11:07:24 INFO: Loading: ner
2024-09-17 11:07:25 INFO: Done loading processors!


In [2]:
def get_constituency_parsing(sentence):
    doc = nlp_dependency(sentence)
    sentence_constituency = str(doc.sentences[0].constituency)
    
    return sentence_constituency


#1.0 NP→PN
def NP_PN(t,vector):
    if t.label() == 'NP' and len(t) >= 1:
        if t[0].label() == 'PN':
            vector[0] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_PN(subtree,vector)

# 4.0 NP→DP NP
def NP_DP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'DP' and t[1].label() == 'NP':
            vector[1] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_DP_NP(subtree,vector)

# 6.2 DP→DT
def DP_DT(t,vector):
    if t.label() == 'DP' and len(t) >= 1:
        if t[0].label() == 'DT':
            vector[2] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            DP_DT(subtree,vector)

#6.6 IP→NP VP PU
def IP_NP_VP_PU(t,vector):
    if t.label() == 'IP' and len(t) >= 3:
        if t[0].label() == 'NP' and t[1].label() == 'VP' and t[2].label() == 'PU':
            vector[3] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            IP_NP_VP_PU(subtree,vector)

#6.8 PRN→PU NP PU
def PRN_PU_NP_PU(t,vector):
    if t.label() == 'PRN' and len(t) >= 3:
        if t[0].label() == 'PU' and t[1].label() == 'NP' and t[2].label() == 'PU':
            vector[4] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            PRN_PU_NP_PU(subtree,vector)

#6.8 NP→NR
def NP_NR(t,vector):
    if t.label() == 'NP' and len(t) >= 1:
        if t[0].label() == 'NR':
            vector[5] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NR(subtree,vector)

#10.0 CP_ADVP_IP
def CP_ADVP_IP(t,vector):
    if t.label() == 'CP' and len(t) >= 2:
        if t[0].label() == 'ADVP' and t[1].label() == 'IP':
            vector[6] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            CP_ADVP_IP(subtree,vector)

#10.6 NP_DNP_NP
def NP_DNP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'DNP' and t[1].label() == 'NP':
            vector[7] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_DNP_NP(subtree,vector)
            
#16.4 ADVP_CS
def ADVP_CS(t,vector):
    if t.label() == 'ADVP' and len(t) >= 1:
        if t[0].label() == 'CS':
            vector[8] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            ADVP_CS(subtree,vector)

#16.8 DNP_NP_DEG
def DNP_NP_DEG(t,vector):
    if t.label() == 'DNP' and len(t) >= 2:
        if t[0].label() == 'NP' and t[1].label() == 'DEG': 
            vector[9] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            DNP_NP_DEG(subtree,vector)

def NP_QP_DNP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'QP' and t[1].label() == 'DNP' and t[2].label() == 'NP':
            vector[10] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_QP_DNP_NP(subtree,vector)

def NP_NP_PRN(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'NP' and t[1].label() == 'PRN':
            vector[11] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NP_PRN(subtree,vector)

def NP_NR_CC_NR(t,vector):
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'NR' and t[1].label() == 'CC' and t[2].label() == 'NR':
            vector[12] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NR_CC_NR(subtree,vector)

def NP_NP_CC_NP(t,vector):
    global score, count
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'NP' and t[1].label() == 'CC' and t[2].label() == 'NP':
            vector[13] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NP_CC_NP(subtree,vector)
            
def get_feature_vector(sentence,vector):
    
    sentence_constituency = get_constituency_parsing(sentence)
    #CFGR features
    tree = Tree.fromstring(sentence_constituency)
    find_NP_PN = NP_PN(tree,vector)
    find_NP_DP_NP = NP_DP_NP(tree,vector)
    find_DP_DT = DP_DT(tree,vector)
    find_IP_NP_VP_PU = IP_NP_VP_PU(tree,vector)
    find_PRN_PU_NP_PU = PRN_PU_NP_PU(tree,vector)
    find_NP_NR = NP_NR(tree,vector)
    find_CP_ADVP_IP = CP_ADVP_IP(tree,vector)
    find_NP_DNP_NP = NP_DNP_NP(tree,vector)
    find_ADVP_CS = ADVP_CS(tree,vector)
    find_DNP_NP_DEG = DNP_NP_DEG(tree,vector)
    #NP features
    find_NP_QP_DNP_NP = NP_QP_DNP_NP(tree,vector)
    find_NP_NP_PRN = NP_NP_PRN(tree,vector)
    find_NP_NR_CC_NR = NP_NR_CC_NR(tree,vector)
    find_NP_NP_CC_NP = NP_NP_CC_NP(tree,vector)

    return vector



In [3]:
def get_dependency_parsing(sentences):
    parsed_words = []
    for sentence in sentences:
        sentence = sentence.strip()  # 去除多余的空白字符
        if not sentence:
            continue
        doc_dependency = nlp_dependency(sentence)
        parsed_words.append(doc_dependency)



In [4]:

def get_feature_vector_dep(sentence,vector):
    doc_dependency = nlp_dependency(sentence)
    a = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','我')
    b = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','将')
    c = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','他')
    d = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','该')
    e = depTripleFuncLex(doc_dependency.sentences[0].words,'NR','case','的')
    f = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','他们')
    g = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','她')
    h = depTripleFuncLex(doc_dependency.sentences[0].words,'他','case','的')
    i = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','nmod:assmod','他')
    j = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','punct','。')
    k = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','但是')
    l = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','你')
    m = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','如果')
    n = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','mark','的')
    o = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','任何')
    p = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','case','因为')
    q = depTripleFuncLex(doc_dependency.sentences[0].words,'NR','cc','和')
    r = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','那些')
    s = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','它')
    t = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','dobj','它')
    if a:
        vector[14] = 1
    if b:
        vector[15] = 1
    if c:
        vector[16] = 1
    if d:
        vector[17] = 1
    if e:
        vector[18] = 1
    if f:
        vector[19] = 1
    if g:
        vector[20] = 1
    if h:
        vector[21] = 1
    if i:
        vector[22] = 1
    if j:
        vector[23] = 1
    if k:
        vector[24] = 1
    if l:
        vector[25] = 1
    if m:
        vector[26] = 1
    if n:
        vector[27] = 1
    if o:
        vector[28] = 1
    if p:
        vector[29] = 1
    if q:   
        vector[30] = 1
    if r:
        vector[31] = 1
    if s:
        vector[32] = 1
    if t:
        vector[33] = 1
    return vector


In [7]:

file = open('processed/cluster0_better_translation.json', 'r', encoding='utf-8')
data = json.load(file)
vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]



# for i,dic in tqdm(enumerate(data), ):
result = []
for i, dic in tqdm.tqdm(enumerate(data), total=len(data)):
    sentence = data[i]
    tmp = get_feature_vector(sentence,vector)
    tmp = get_feature_vector_dep(sentence,vector)
    str_tmp = ''.join(str(x) for x in tmp)
    vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    result.append({'zh':sentence, 'vec':str_tmp})

with open('processed/cluster0_better_transfvec.json','w', encoding='utf-8') as f:
    json.dump(result,f,ensure_ascii=False)


100%|██████████| 695/695 [02:56<00:00,  3.94it/s]


TypeError: Object of type set is not JSON serializable

In [11]:
with open('processed/2024-09-14-21-42-57_processed.json','w', encoding='utf-8') as f:
    json.dump(data,f,ensure_ascii=False)